In [1]:
import fitz
import os
import json
import shutil
import re
import pandas as pd
import numpy as np
from functools import wraps
import time
import re
import matplotlib.pyplot as plt
import warnings

import pandas as pd
import numpy as np
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm







def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        # print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        print(f'Function {func.__name__} Took {total_time:.4f} seconds',end='\r')
        return result
    return timeit_wrapper

def is_valid_path(path):
    try:
        # If the path is absolute or can be made absolute, it's considered a valid path
        absolute_path = os.path.abspath(path)
        return os.path.exists(absolute_path)
    except OSError:
        return False

    
os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner'

In [ ]:
# !python -m spacy download es_dep_news_trf

# load

## data

In [7]:
base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'
if is_valid_path(base_path):
    df = pd.read_pickle(os.path.join(base_path,'result_df2.pkl'))

In [2]:
base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'
df = pd.read_pickle(os.path.join(base_path,'result_df4.pkl'))

In [31]:
df.chapter.unique()

array(['contratos_sobre_personas_jurídicas',
       'convocatorias_y_avisos_comerciales', 'edictos_judiciales',
       'partidos_políticos', 'balances'], dtype=object)

Supuesto: los estatutos tienen que declararse en la sección 'contratos_sobre_personas_jurídicas' y llevan al menos 150 tokens. 

In [4]:
estatutos_ls = df[(df.chapter=='contratos_sobre_personas_jurídicas')&(df.word_count>=150)].text.tolist()
len(estatutos_ls)

59111

## modelo

In [5]:
ner_model_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner\ner_model_1' # ner_model_1
# ner_model_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner\textcat_model_1'

nlp_ner = spacy.load(os.path.join(ner_model_path,'model-best'))

In [6]:
@timeit
def get_ner(text):
    doc = nlp_ner(text)
    res =[]
    
    for ent in doc.ents:
        res.append((ent.label_,ent.text))
        
    # Initialize an empty dictionary
    result_dict = {}

    # Loop through the tuples and populate the dictionary
    for key, value in res:
        if key in result_dict:
            # If the key already exists, append the value to the existing list
            result_dict[key].append(value)
        else:
            # If the key is encountered for the first time, create a new list with the value
            result_dict[key] = [value]     
        
    required_keys = ['SOCIO', 'DNI', 'ESCRIBANO']
    for key in required_keys:
        result_dict.setdefault(key, [])    
    return result_dict

In [30]:
estatutos_ls[5100]

'Constituida por Esc. Nº 348 del 13/10/2017 por ante el Registro 1177 de C.A.B.A.- Socios: Fabricio Mariano TKACZYK, argentino, nacido el 28/01/1986, D.N.I.: 32.037.644, soltero, abogado, con domicilio en Sanabria 3695, 3° Piso, de C.A.B.A.; y Daniel Guillermo D’ALMEIDA, argentino, nacido el 09/12/1954, D.N.I.: 11.488.391, divorciado de sus primeras nupcias de María Inés Álvarez, empresario, con domicilio en Olavarría 3679, Caseros, Pdo. Tres de Febrero, Pcia. de Bs. As.- 1) Denominación: “WORK CLOTHES\xa0S.R.L.”.- 2) Duración: 99 años.- 3) Objeto: brindar por cuenta propia, y/o de terceros y/o asociada a terceros la confección, producción, venta, comercialización, importación y/o exportación de tejidos, ropa y uniformes para empresas y afines. A tal fin la sociedad tiene plena capacidad jurídica para adquirir derechos, contraer obligaciones y ejercer los actos que no sean prohibidos por las leyes o por este contrato.- 4) Capital: $\xa0100.000 dividido en 1.000 de cuotas de 100 pesos v

In [35]:
get_ner(df[df.title=='BETHEL WORLD\xa0S.A.'].text.tolist()[0])

{'SOCIO': ['Rolando IRIARTE PIEDRA',
  'Eduviges TICONA ALVAREZ',
  'Karen Nilda NINAVIA VASQUEZ',
  'Genaro MAMANI TITO',
  'Lucy Celia AMAYA FERNANDEZ',
  'Melanio Ramón SILVERO',
  'Jhonny Eduardo MAMANI HEREDIA',
  'Richard Ramón SEJAS FERNÁNDEZ',
  'Juan Carlos VIGABRIEL CAMACHO',
  'Néstor Alberto BARRA',
  'Karina Ester CALERO',
  'Wilder Rubén VIDAL FERNANDEZ',
  'Ana María GARCÍA QUIROGA',
  'Giovanna VILLCA ABREGO',
  'Ulises Abel VELASQUEZ',
  'Alejandro Fabián CALERO',
  'Graciela Noemí CASTILLO',
  'Nestor Osvaldo CALERO',
  'Jesús VIGABRIEL CAMACHO',
  'Héctor Alejandro CAGUASIRI',
  'ROSSEL, Gustavo Sixto',
  'Zacarías LIPA VEGA',
  'Roberto Martin CAGUASIRI'],
 'DNI': ['92.849.692',
  '92.931.282',
  '94.416.664',
  '93.970.149',
  '94.853.779',
  '17.067.835',
  '92.950.005',
  '92.110.545',
  '94.838.987',
  '28.161.836',
  '21.551.847',
  '94.078.384',
  '94.000.668',
  '94.299.738',
  '28.123.929',
  '24.379.336',
  '26.439.437',
  '41.151.684',
  '93.650.446',
  '3

In [29]:
get_ner(estatutos_ls[5100])

{'SOCIO': ['Fabricio Mariano TKACZYK', 'Daniel Guillermo D’ALMEIDA'],
 'DNI': ['32.037.644', '11.488.391'],
 'ESCRIBANO': ['Omar Bogado']}

# PREPROCESAMIENTO COMPLEMENTARIO

In [15]:
df[df.id=='seccion_segunda_20181210.pdfcontratos_sobre_personas_jurídicassociedades_anónimasBANCO HIPOTECARIO\xa0S.A.']

,index,file,page,font,size,text,chapter,subchapter,title,word_count,SOCIO,DNI,ESCRIBANO,SOCIEDAD,id,consecutive_count,consecutive_group
341222,48,seccion_segunda_20181210.pdf,2,HelveticaNeueLTStd-Roman,10.0,Comunica que la Asamblea Extraordinaria de Acc...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,520,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,1.0,129054
341224,50,seccion_segunda_20181210.pdf,3,HelveticaNeueLTStd-Roman,10.0,"en el país o en el exterior, dentro de los lím...",contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1096,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,3.0,129054
341226,52,seccion_segunda_20181210.pdf,4,HelveticaNeueLTStd-Roman,10.0,que lo adquieran por encontrarse la Sociedad e...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1094,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,5.0,129054
341228,54,seccion_segunda_20181210.pdf,5,HelveticaNeueLTStd-Roman,10.0,"(A) La identidad, nacionalidad, domicilio y nú...",contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1164,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,7.0,129054
341230,56,seccion_segunda_20181210.pdf,6,HelveticaNeueLTStd-Roman,10.0,régimen de la oferta pública de adquisición. S...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1113,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,9.0,129054
341232,58,seccion_segunda_20181210.pdf,7,HelveticaNeueLTStd-Roman,10.0,"originales, por omisión de ejercicio o por ces...",contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1096,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,11.0,129054
341234,60,seccion_segunda_20181210.pdf,8,HelveticaNeueLTStd-Roman,10.0,aún dentro de la clase con arreglo a las previ...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1087,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,13.0,129054
341236,62,seccion_segunda_20181210.pdf,9,HelveticaNeueLTStd-Roman,10.0,Impuesto que haya arrojado el Ejercicio Económ...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1051,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,15.0,129054
341238,64,seccion_segunda_20181210.pdf,10,HelveticaNeueLTStd-Roman,10.0,judiciales y administrativas y asumir el papel...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1073,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,17.0,129054
341240,66,seccion_segunda_20181210.pdf,11,HelveticaNeueLTStd-Roman,10.0,Comité Ejecutivo: I) Conducir la gestión ordin...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BANCO HIPOTECARIO S.A.,1079,[],[],[],NaN,seccion_segunda_20181210.pdfcontratos_sobre_pe...,19.0,129054


In [50]:
df.subchapter.unique()

array([nan, 'buenos_aires', 'sociedades_anónimas',
       'sociedades_de_responsabilidad_limitada', 'convocatorias',
       'avisos_comerciales', 'remates_comerciales',
       'citaciones_y_notificaciones._concursos_y_quiebras._otros',
       'sucesiones', 'remates_judiciales',
       'transferencias_de__fondo_de_comercio_',
       'sociedades_en_comandita_por_acciones', 'otras_sociedades',
       'balances', 'sociedades_por_acción_simplificada',
       'partidos_políticos'], dtype=object)

In [49]:
df[df.title=='BANCO HIPOTECARIO S.A.']

,index,file,page,font,size,text,chapter,subchapter,title,word_count,SOCIO,DNI,ESCRIBANO,SOCIEDAD,id
613898,533,seccion_segunda_20200317.pdf,64,HelveticaNeueLTStd-Roman,10.00000,Domicilio Legal Reconquista 151 – Ciudad Autón...,convocatorias_y_avisos_comerciales,balances,BANCO HIPOTECARIO S.A.,41,NN,NN,NN,NN,seccion_segunda_20200317.pdfconvocatorias_y_av...
613899,534,seccion_segunda_20200317.pdf,64,HelveticaNeueLTStd-Roman,8.53815,En milesCantidad de AccionesSuscriptoIntegrado...,convocatorias_y_avisos_comerciales,balances,BANCO HIPOTECARIO S.A.,16,NN,NN,NN,NN,seccion_segunda_20200317.pdfconvocatorias_y_av...
613900,535,seccion_segunda_20200317.pdf,64,HelveticaNeueLTStd-Roman,10.00000,NOTA: La documentación que integra este Balanc...,convocatorias_y_avisos_comerciales,balances,BANCO HIPOTECARIO S.A.,20,NN,NN,NN,NN,seccion_segunda_20200317.pdfconvocatorias_y_av...
613901,536,seccion_segunda_20200317.pdf,65,HelveticaNeueLTStd-LtCn,10.00000,BOLETÍN OFICIAL Nº 34.331 - Segunda Sección 6...,convocatorias_y_avisos_comerciales,balances,BANCO HIPOTECARIO S.A.,16,NN,NN,NN,NN,seccion_segunda_20200317.pdfconvocatorias_y_av...
613902,537,seccion_segunda_20200317.pdf,65,HelveticaNeueLTStd-Bd,17.00000,EDICTOS JUDICIALES,edictos_judiciales,balances,BANCO HIPOTECARIO S.A.,2,NN,NN,NN,NN,seccion_segunda_20200317.pdfedictos_judiciales...
613903,538,seccion_segunda_20200317.pdf,65,HelveticaNeueLTStd-Bd,14.00000,CITACIONES Y NOTIFICACIONES. CONCURSOS Y QUIEB...,edictos_judiciales,citaciones_y_notificaciones._concursos_y_quieb...,BANCO HIPOTECARIO S.A.,7,NN,NN,NN,NN,seccion_segunda_20200317.pdfedictos_judiciales...
613904,539,seccion_segunda_20200317.pdf,65,HelveticaNeueLTStd-Bd,12.00000,NUEVOS,edictos_judiciales,citaciones_y_notificaciones._concursos_y_quieb...,BANCO HIPOTECARIO S.A.,1,NN,NN,NN,NN,seccion_segunda_20200317.pdfedictos_judiciales...


In [14]:
condition = (df.chapter=='contratos_sobre_personas_jurídicas')&(df.word_count>=150)
control_ls = pd.DataFrame(df[condition].title.value_counts())
control_ls[control_ls['count']>1]

,count
title,
BANCO HIPOTECARIO S.A.,17
TELECOM ARGENTINA S.A.,14
KLIMBER S.A.,12
IRSA INVERSIONES Y REPRESENTACIONES S.A.,12
AGUAS DANONE DE ARGENTINA S.A.,11
...,...
MUNDIAL ARGENTINA S.A.,2
UROBORO S.R.L.,2
WORKSPACES S.A.,2


In [38]:
control_ls[control_ls['count']>1].index[-10:]

Index(['ABRUMARGA S.A.', 'SWAHILI BAMBU LATAM S.A.S.', 'TERCER SOLDADO S.R.L.',
       'ELOVA S.A.', 'SARMIENTO 2302 S.A.', 'MUNDIAL ARGENTINA S.A.',
       'UROBORO S.R.L.', 'WORKSPACES S.A.', 'NISSAN ARGENTINA S.A.',
       'MAUS SOLUTIONS S.R.L.'],
      dtype='object', name='title')

In [36]:
len(control_ls[control_ls['count']>1])

6370

In [13]:
pd.DataFrame(control_ls)

,count
title,
BANCO HIPOTECARIO S.A.,17
TELECOM ARGENTINA S.A.,14
KLIMBER S.A.,12
IRSA INVERSIONES Y REPRESENTACIONES S.A.,12
AGUAS DANONE DE ARGENTINA S.A.,11
...,...
DATAKI S.A.S.,1
DULCE Y SALADO BAVE S.A.S.,1
ELIFESTYLE S.A.S.,1


In [9]:
df.title.value_counts()

title
JUZGADOS NACIONALES EN LO CIVILPublicación extractada (Acordada Nº 41/74 C.S.J.N.)              96669
e-mail: dnro@boletinoficial.gob.ar                                                              32035
BANCO DE SAN JUAN S.A.                                                                          10133
NUEVO BANCO DE ENTRE RIOS S.A.                                                                   5974
e-mail: dnro@boletinoficial.gob.arRegistro Nacional de la Propiedad Intelectual Nº 5.218.874     5173
                                                                                                ...  
RECOLETA BREWPUB S.A.                                                                               1
SENSOTEC S.A.                                                                                       1
ROCABREN S.A.                                                                                       1
SABOR MIX S.R.L.                                                            

In [172]:
# Identify rows where consecutive titles are the same
mask = (df['title'] == df['title'].shift(1))&(df.chapter=='contratos_sobre_personas_jurídicas')&(df.word_count>=150)

# Combine 'text' values for consecutive rows with the same title
df.loc[mask, 'text'] = df['text'].shift(1) + ' ' + df['text']

# Drop duplicated rows
df = df[~mask]

# Reset index if needed
df = df.reset_index(drop=True)

In [44]:
condition = (df.chapter=='contratos_sobre_personas_jurídicas')&(df.word_count>=150)
len(df[condition])

64939

In [55]:
len(df[df.word_count>150])

157303

In [15]:
# Create a new DataFrame with the first value in each group
result_df = df.groupby('consecutive_group').first().reset_index()

# Merge the 'text' values separated by whitespace within each group
result_df['text'] = df.groupby('consecutive_group')['text'].agg('  '.join).reset_index()['text']

# Drop the temporary column if not needed
result_df = result_df.drop(columns=['consecutive_group'])

In [16]:
len(result_df)

335016

In [181]:
# result_df[result_df.chapter=='contratos_sobre_personas_jurídicas'].title.unique().tolist()

In [28]:
result_df[result_df.title=='e-mail: dnro@boletinoficial.gob.ar'].text.unique()

array(['SOCIEDADES ANÓNIMAS', 'SOCIedAdeS AnónIMAS',
       'SOCIEDADES ANÓNIMAS  NUEVOS', 'CONVOCATORIAS  NUEVOS',
       'SOCIEDADES POR ACCIÓN SIMPLIFICADA',
       'SOCIEDADES DE RESPONSABILIDAD LIMITADA',
       'SOCIEDADES ANÓNIMAS   NUEVOS'], dtype=object)

In [17]:
len(result_df[result_df.title=='e-mail: dnro@boletinoficial.gob.ar'])
result_df = result_df[result_df.title!='e-mail: dnro@boletinoficial.gob.ar']

In [30]:
result_df['word_count'] = [ len(re.split(r'[ \.,]+', t)) for t in result_df['text']]

In [ ]:
result_df['word_count'].hist()

In [187]:
result_df.columns

Index(['index', 'file', 'page', 'font', 'size', 'text', 'chapter',
       'subchapter', 'title', 'word_count', 'SOCIO', 'DNI', 'ESCRIBANO',
       'SOCIEDAD', 'id', 'consecutive_count'],
      dtype='object')

In [93]:
# Identify the start of a new consecutive group
mask = df['id'] != df['id'].shift(1)
df['consecutive_group'] = mask.cumsum()

In [94]:
len(df[df['consecutive_group']==0])

0

In [92]:
control_ls = pd.DataFrame(df[condition].id.value_counts())
control_ls[control_ls['count']>1]

C:\Users\juani\AppData\Local\Temp\ipykernel_6692\4068162526.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control_ls = pd.DataFrame(df[condition].id.value_counts())


,count
id,
seccion_segunda_20181210.pdfcontratos_sobre_personas_jurídicassociedades_anónimasBANCO HIPOTECARIO S.A.,12
seccion_segunda_20220801.pdfcontratos_sobre_personas_jurídicassociedades_anónimasAGUAS DE ORIGEN S.A.,7
seccion_segunda_20190703.pdfcontratos_sobre_personas_jurídicassociedades_anónimasCT BARRAGAN S.A.,7
seccion_segunda_20180105.pdfcontratos_sobre_personas_jurídicassociedades_anónimasFUTUROS Y OPCIONES.COM S.A.,6
seccion_segunda_20220706.pdfcontratos_sobre_personas_jurídicassociedades_anónimasGLENCORE PACHON S.A.,6
...,...
seccion_segunda_20191120.pdfcontratos_sobre_personas_jurídicassociedades_anónimasPE EL CARRIZAL S.A.,2
seccion_segunda_20170602.pdfcontratos_sobre_personas_jurídicassociedades_anónimasTELECOM ARGENTINA S.A.,2
seccion_segunda_20191015.pdfcontratos_sobre_personas_jurídicassociedades_anónimasQUARK NETWORKS S.A.,2


In [59]:
control_ls[control_ls['count']>1].index.tolist()[0]

'seccion_segunda_20181210.pdfcontratos_sobre_personas_jurídicassociedades_anónimasBANCO HIPOTECARIO\xa0S.A.'

In [57]:
# df[(df.ESCRIBANO=='NN')&(df.word_count>150)].text.tolist()

In [46]:
df[condition].ESCRIBANO.value_counts()

ESCRIBANO
[]                                                  31606
[Trámites a Distancia]                              10726
NN                                                   5828
[Omar Bogado]                                         469
[Daniel Ricoso]                                       275
                                                    ...  
[Carmen Spatola]                                        1
[Luisa Rodenas Fernandez]                               1
[JUAN SALAS, JUAN SALAS]                                1
[catalina benavidez TIGRE, Trámites a Distancia]        1
[JUAREZ DOVER]                                          1
Name: count, Length: 2147, dtype: int64

In [56]:
# df[condition].text.tolist()[0:20]

# ejecucion

### time control

In [9]:
start = time.time()
r = [*map(get_ner,estatutos_ls[0:1000])]
t = np.round(time.time()-start,1)
print(f'final runtime execution for this:  {t} seconds' )

final runtime execution for this:  65.1 seconds


In [10]:
len(estatutos_ls),len(df)

(59111, 333984)

In [11]:
f'estimated runtime of {np.round(len(estatutos_ls)*t/1000/60,1)} minutes'

'estimated runtime of 64.1 minutes'

### real execution

In [12]:
start = time.time()
r = [*map(get_ner,estatutos_ls)]
t = np.round(time.time()-start,1)
print(f'final runtime execution for this:  {t} seconds' )

final runtime execution for this:  3956.4 seconds


In [37]:
len(r)

59111

In [22]:
r[-1]

{'SOCIO': ['Santiago Nicolas SANCHEZ BUSTOS',
  'Facundo Matias SANCHEZ BUSTOS',
  'Carolina Alejandra BUSTOS'],
 'DNI': ['40.010.290', '42.821.315', '27.085.928'],
 'ESCRIBANO': []}

In [36]:
estatutos_ls[-1]

'1) 14/07/2023. 2) Santiago Nicolas SANCHEZ BUSTOS, argentino, soltero, empresario, 04/11/1996, DNI 40.010.290, CUIT 20-40010290-3, Bermudez 1726, CABA; Facundo Matias SANCHEZ BUSTOS, argentino, soltero, empresario, 03/11/2000, DNI 42.821.315, CUIT 20-42821315-8; y Carolina Alejandra BUSTOS, argentina, casada, empresaria, 22/01/1979, DNI 27.085.928, CUIT. 27-27085928-9, ambos domiciliados en Bermudez 2113, CABA. 3) THE TOY STORE\xa0S.R.L. 4) Sede Social: Bermúdez 1726, CABA. 5) 30 años a partir de su inscripción en R.P.C. 6) OBJETO: Fabricación, distribución y venta al por mayor y al por menor de artículos de indumentaria, calzado, marroquinería, juguetería, blanquería, librería y accesorios. 7) 1.000.000 Pesos. 100.000 cuotas de $\xa010 pesos, valor nominal cada una. Suscripción: Santiago Nicolas SANCHEZ BUSTOS y Facundo Matias SANCHEZ BUSTOS: 33.333 cuotas ($\xa0333.330.-) cada uno; y Carolina Alejandra BUSTOS, suscribe 33.334 cuotas ($\xa0333.340.-). Integración: 25% de las respecti

In [23]:
r[0:3]

[{'SOCIO': ['Emilio Adrián LEIVA', 'Daniel Horacio SÁNCHEZ', 'Majed MOHANNA'],
  'DNI': ['26007027', '25747237', '18886526'],
  'ESCRIBANO': ['María Cardoso']},
 {'SOCIO': ['Julián Pujol', 'Orlando Rubén Montanari'],
  'DNI': ['26561990', '13266283'],
  'ESCRIBANO': ['Martín Esposito']},
 {'SOCIO': ['Ana Mayra CATICHA', 'Martin NICOLA'],
  'DNI': ['22.143.500', '22.888.888'],
  'ESCRIBANO': []}]

In [17]:
df_to_add = pd.DataFrame(r)
df_to_add.head()

,SOCIO,DNI,ESCRIBANO,SOCIEDAD
0,"[Emilio Adrián LEIVA, Daniel Horacio SÁNCHEZ, ...","[26007027, 25747237, 18886526]",[María Cardoso],NaN
1,"[Julián Pujol, Orlando Rubén Montanari]","[26561990, 13266283]",[Martín Esposito],NaN
2,"[Ana Mayra CATICHA, Martin NICOLA]","[22.143.500, 22.888.888]",[],NaN
3,"[Rolando IRIARTE PIEDRA, Eduviges TICONA ALVAR...","[92.849.692, 92.931.282, 94.416.664, 93.970.14...",[EDUARDO OLIVERA],NaN
4,"[Sergio Francisco Schwarzkopf, Nelson Mario Lu...",[24142579],[],NaN


In [18]:
# ASIGNAMOS LOS VALORES: 

df['SOCIO'] = 'NN'
df['DNI'] = 'NN'
df['ESCRIBANO'] = 'NN'
df['SOCIEDAD'] = 'NN'

condition = (df.chapter=='contratos_sobre_personas_jurídicas')&(df.word_count>=150)

df.loc[condition,'SOCIO'] = df_to_add['SOCIO'].values
df.loc[condition,'DNI'] = df_to_add['DNI'].values
df.loc[condition,'ESCRIBANO'] = df_to_add['ESCRIBANO'].values
df.loc[condition,'SOCIEDAD'] = df_to_add['SOCIEDAD'].values

In [116]:
664749 - 597795

66954

In [38]:
df[df.DNI!='NN'].ESCRIBANO.tolist()[2]

[]

In [138]:
df[df.ESCRIBANO!='NN'].text.tolist()[2]

'Constitución: Por escritura 71 del 15/04/17 Rº 4 del Partido de Pehuajó: 1) Socios: Los cónyuges en primeras nupcias Ana Mayra CATICHA, 02/07/71, DNI 22.143.500, CUIT 27-22143500-7; y Martin NICOLA, 22/12/72, DNI 22.888.888, CUIT 20-22888888-6, ambos argentinos, comerciantes y domiciliados en Elizon 790 Localidad y Partido de Pehuajó, Prov. Bs As. 2) BAZAR MEGA\xa0S.A. 3) 99 años. 4) $\xa0100.000. 5) Objeto: a) Comerciales: Mediante la compra, venta, permuta, consignación, importación y exportación de todo tipo de productos de juguetería, cotillón, librería, kiosco, bazar, tienda, bijouterie, electrónica, relojería, óptica, iluminación, decoración, artículos para la caza y pesca deportiva y herramientas en general; comercialización de los mismos y de los frutos y productos derivados de ellos; b) Inmobiliaria: Mediante la compraventa, arrendamiento y administración de inmuebles urbanos y rurales, la subdivisión de tierras y su urbanización, como asimismo todas las operaciones comprendi

In [39]:
len(df)-len(df[df.SOCIO=='NN'])

59111

In [40]:
df.ESCRIBANO.value_counts()

ESCRIBANO
NN                                                  274873
[]                                                   31606
[Trámites a Distancia]                               10726
[Omar Bogado]                                          469
[Daniel Ricoso]                                        275
                                                     ...  
[Carmen Spatola]                                         1
[Luisa Rodenas Fernandez]                                1
[JUAN SALAS, JUAN SALAS]                                 1
[catalina benavidez TIGRE, Trámites a Distancia]         1
[JUAREZ DOVER]                                           1
Name: count, Length: 2147, dtype: int64

In [25]:
esc_count = pd.DataFrame(df.ESCRIBANO.value_counts())

In [26]:
len(esc_count),len(esc_count[esc_count['count']>1]),len(esc_count[esc_count['count']>2]),len(esc_count[esc_count['count']>1]),len(esc_count[esc_count['count']>3])

(2147, 1215, 965, 1215, 818)

In [27]:
df[condition][3:5].title.tolist()

['BETHEL WORLD\xa0S.A.', 'BIGPAL\xa0S.A.']

In [28]:
for t in df[df.title=='BETHEL WORLD\xa0S.A.'].text:
    print(t)
    print('---')

Por escritura 155 Folio 419 del 22/05/2017 se constituyó la sociedad. Socios: 1) Rolando IRIARTE PIEDRA, soltero, boliviano, 23/06/1978, DNI 92.849.692, domicilio real y especial en la calle Esteban Bonorino Edificio 26 S/N, Piso 1, departamento A, Barrio Rivadavia, CABA. 2) Eduviges TICONA ALVAREZ, soltera, boliviana, 17/10/1969, DNI 92.931.282, domicilio real en la Avenida Olivera 1463, Planta Baja, Unidad Funcional 2, CABA. 3) Karen Nilda NINAVIA VASQUEZ, soltera, boliviana, 14/04/1978, DNI 94.416.664, domicilio real en Barrio Rivadavia 1 N° 408, CABA; 4) Genaro MAMANI TITO, soltero, boliviano, 12/07/1976, DNI 93.970.149, domicilio real en la calle José Camilo Paz N° 3050, Planta Baja, departamento “C”, CABA. 5) Lucy Celia AMAYA FERNANDEZ, soltera, peruana, 10/07/1983, DNI 94.853.779, con domicilio real en la Manzana 29, Casa 268 S/N, Villa 1 11 14, CABA; 6) Melanio Ramón SILVERO, argentino, casado en primeras nupcias con Carmen Beatriz Rojas, 22/10/1963, DNI 17.067.835, con domicil

### limpiamos filas innecesarias

In [4]:
print(len(df))

# eliminamos los encabezados:
filtered_rows = df[df['text'].str.startswith('BOLETÍN OFICIAL Nº')]
print(len(filtered_rows))
df = df[~df['text'].str.startswith('BOLETÍN OFICIAL Nº')]


# eliminamos los encabezados:
filtered_rows = df[df['text'].str.startswith(' BOLETÍN OFICIAL Nº')]
print(len(filtered_rows))
df = df[~df['text'].str.startswith(' BOLETÍN OFICIAL Nº')]


# eliminamos los pies de página:

filtered_rows = df[df['subchapter'].isna()]
print(len(filtered_rows))
df = df[~df['subchapter'].isna()]

df['subchapter'] = df.subchapter.astype(str)
filtered_rows = df[df['subchapter']=='buenos_aires']
print(len(filtered_rows))
df = df[df['subchapter']!='buenos_aires']




# Display the result
print(len(df))

664749
308
112547
11789
28578
511527


### CORREGIMOS TEXTOS CORTADOS

In [5]:
# creamos columna con id unívoco:
df['id'] =  df['file'] + df['chapter'] + df['subchapter'] + df['title']

In [6]:
df['consecutive_count'] = (df['id'] != df['id'].shift()).cumsum()
# Count occurrences within each group
df['consecutive_count'] = df.groupby(['id', 'consecutive_count']).cumcount() + 1

In [7]:
# Identify the start of a new consecutive group
mask = df['id'] != df['id'].shift(1)
df['consecutive_group'] = mask.cumsum()

In [8]:
len(df[df['consecutive_count']>1])

176511

In [9]:
condition = (df.chapter=='contratos_sobre_personas_jurídicas')&(df.consecutive_count>1)
len(df[condition])

15782

In [10]:
condition = (df.chapter!='contratos_sobre_personas_jurídicas')&(df.consecutive_count>1)
len(df[condition])
sel2_ls = df[condition].consecutive_group.unique().tolist()
len(sel2_ls)

43790

In [11]:
# for t in df[df.consecutive_group==sel2_ls[15]].text:
#     print(t)
#     print('---')

In [12]:
sel_ls = df[condition].consecutive_group.unique().tolist()


In [13]:
for t in df[df.consecutive_group==sel_ls[2]].text:
    print(t)
    print('---')

CONVOCATORIA A LAS ASAMBLEAS ELECTORALES DE DISTRITO DE GRAN COOPERATIVA DE CRÉDITO, VIVIENDA, CONSUMO Y SERVICIOS SOCIALES LIMITADA. En virtud de lo dispuesto por el 50 de la Ley 20.337, y según surge del Reglamento Electoral de Distrito de Gran Cooperativa de Crédito, Vivienda, Consumo y Servicios Sociales Limitada, que fuera aprobado por Resolución INAES Nº 4.984 del 14 de diciembre de 2.009 e inscripto al Folio 224 del Libro 7º bajo Acta Nº 4.050 con fecha 4 de enero de 2.010, el Consejo de Administración de GRAN COOPERATIVA DE CRÉDITO, VIVIENDA, CONSUMO Y SERVICIOS SOCIALES LIMITADA convoca a los asociados incluidos en el padrón correspondiente al último ejercicio cerrado al 31/03/2017, debidamente actualizado, a ejercer su derecho a voto a las Asambleas Electorales de Distrito de Gran Cooperativa de Crédito, Vivienda, Consumo y Servicios Sociales Limitada, a realizarse los días 14 y 15 de Junio de 2017 a fin de considerar el siguiente orden del día: “Orden del Día: 1.- Lectura y 

In [24]:
df[condition].title.unique()[0]

'WOOFI\xa0S.R.L.'

In [14]:
df[df.title=='BETHEL WORLD\xa0S.A.']

,index,file,page,font,size,text,chapter,subchapter,title,word_count,SOCIO,DNI,ESCRIBANO,SOCIEDAD,id,consecutive_count,consecutive_group
40,40,seccion_segunda_20170601.pdf,2,HelveticaNeueLTStd-Roman,10.0,Por escritura 155 Folio 419 del 22/05/2017 se ...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BETHEL WORLD S.A.,648,"[Rolando IRIARTE PIEDRA, Eduviges TICONA ALVAR...","[92.849.692, 92.931.282, 94.416.664, 93.970.14...",[],NaN,seccion_segunda_20170601.pdfcontratos_sobre_pe...,1,6
42,42,seccion_segunda_20170601.pdf,3,HelveticaNeueLTStd-Roman,10.0,Domicilio social: CABA. Sede Social: Carhue 45...,contratos_sobre_personas_jurídicas,sociedades_anónimas,BETHEL WORLD S.A.,488,[],[],[EDUARDO OLIVERA],NaN,seccion_segunda_20170601.pdfcontratos_sobre_pe...,2,6


### agrupamos

### GUARDAMOS

In [19]:
len(result_df)

333984

In [144]:
base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'
if is_valid_path(base_path):
    df.to_pickle(os.path.join(base_path,'result_df3.pkl'))

In [155]:
base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'
if is_valid_path(base_path):
    df[condition].to_pickle(os.path.join(base_path,'df_for_labeling.pkl'))

In [18]:
base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'
if is_valid_path(base_path):
    result_df.to_pickle(os.path.join(base_path,'result_df4.pkl'))

In [43]:
base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'
if is_valid_path(base_path):
    df.to_pickle(os.path.join(base_path,'result_df5.pkl'))

#### GUARDAMOS BASE PARA ANOTAR

In [71]:
# generamos un dataset
base_ls = df[condition].text[0:500].tolist()
base_str = '---'.join(base_ls)
# print(base_str)

base_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner'

with open(os.path.join(base_path,'base_str_500.txt'), "w", encoding="utf-8") as text_file:
    text_file.write(base_str)

In [62]:
base_ls = df[condition].text[0:500].tolist()

In [68]:
len(base_ls)

500

# EDA SOCIEDADES

In [74]:
path_file_sociedades = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner\001_Named_Entity_Recognition\datos\registro-nacional-sociedades-2022-semestre-2\registro-nacional-sociedades-202211.csv'
soc = pd.read_csv(path_file_sociedades)

In [100]:
soc.columns

Index(['cuit', 'razon_social', 'fecha_hora_contrato_social', 'tipo_societario',
       'fecha_hora_actualizacion', 'numero_inscripcion',
       'dom_fiscal_provincia', 'dom_fiscal_localidad', 'dom_fiscal_calle',
       'dom_fiscal_numero', 'dom_fiscal_piso', 'dom_fiscal_departamento',
       'dom_fiscal_cp', 'dom_fiscal_estado_domicilio', 'dom_legal_provincia',
       'dom_legal_localidad', 'dom_legal_calle', 'dom_legal_numero',
       'dom_legal_piso', 'dom_legal_departamento', 'dom_legal_cp',
       'dom_legal_estado_domicilio'],
      dtype='object')

In [154]:
type(soc.fecha_hora_contrato_social[0])

str

In [ ]:
df[df['emails'].str.contains('gmail')]

In [98]:
+2024-1969, +2024-2011

(55, 13)

In [99]:
55-13

42

In [97]:
mask = soc.razon_social.str.contains('N. Y A. FIDUCIARIA ', na=False)
soc[mask]

,cuit,razon_social,fecha_hora_contrato_social,tipo_societario,fecha_hora_actualizacion,numero_inscripcion,dom_fiscal_provincia,dom_fiscal_localidad,dom_fiscal_calle,dom_fiscal_numero,...,dom_fiscal_cp,dom_fiscal_estado_domicilio,dom_legal_provincia,dom_legal_localidad,dom_legal_calle,dom_legal_numero,dom_legal_piso,dom_legal_departamento,dom_legal_cp,dom_legal_estado_domicilio
702250,3.071205e+10,N. Y A. FIDUCIARIA SRL,2011-09-20-00:00,SOCIEDAD DE RESPONSABILIDAD LIMITADA,2022-09-22-08:05,NaN,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,NUEVA YORK,4074.0,...,1419.0,DECLARADO POR INTERNET,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,MAIPU,687.0,10,NaN,1006.0,DECLARADO


In [79]:
soc[soc.razon_social.str.contains('SEGUROS ')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [75]:
soc.head()

,cuit,razon_social,fecha_hora_contrato_social,tipo_societario,fecha_hora_actualizacion,numero_inscripcion,dom_fiscal_provincia,dom_fiscal_localidad,dom_fiscal_calle,dom_fiscal_numero,...,dom_fiscal_cp,dom_fiscal_estado_domicilio,dom_legal_provincia,dom_legal_localidad,dom_legal_calle,dom_legal_numero,dom_legal_piso,dom_legal_departamento,dom_legal_cp,dom_legal_estado_domicilio
0,3.050000e+10,SEGUROS SURA S.A,1912-04-30-10:43,SOCIEDAD ANONIMA,2022-10-24-10:37,NaN,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,"GRIERSON,CECILIA BOULEVARD",255.0,...,1107.0,CONFIRMADO,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,"GRIERSON,CECILIA BOULEVARD",255.0,1,NaN,1107.0,DECLARADO
1,3.050000e+10,BANCO COMERCIAL DE TRES ARROYOS S A,1906-09-01-00:00,SOCIEDAD ANONIMA,2022-06-14-18:59,NaN,BUENOS AIRES,TRES ARROYOS,BETOLAZA,189.0,...,7500.0,INEXISTENTE - DESCONOCIDO,BUENOS AIRES,TRES ARROYOS,AV MORENO,300.0,NaN,NaN,7500.0,NO CONFIRMADO
2,3.050000e+10,BANCO COMERCIAL DEL TANDIL S A,1964-03-21-00:00,SOCIEDAD ANONIMA,2022-09-24-13:02,NaN,BUENOS AIRES,TANDIL,GRAL PINTO,602.0,...,7000.0,DECLARADO,BUENOS AIRES,TANDIL,GRAL PINTO,602.0,NaN,NaN,7000.0,NO CONFIRMADO
3,3.050000e+10,BOSTON COMPAÑIA ARGENTINA DE SEGUROS S A,1925-02-19-11:00,SOCIEDAD ANONIMA,2022-11-15-13:14,NaN,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,SUIPACHA,268.0,...,1008.0,CONFIRMADO,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,SUIPACHA,268.0,03,NaN,1008.0,DECLARADO
4,3.050000e+10,BANCO DE CREDITO COMERCIAL,1913-03-10-00:00,SOCIEDAD ANONIMA,2022-06-14-18:59,NaN,SANTA FE,SANTA FE,SAN MARTIN,2498.0,...,3000.0,INEXISTENTE - DESCONOCIDO,SANTA FE,SANTA FE,SAN MARTIN,2498.0,NaN,NaN,3000.0,NO CONFIRMADO


In [45]:
all_provinces = soc.tipo_societario.unique().tolist()

soc.groupby('tipo_societario')['cuit'].count().reindex(all_provinces, fill_value=0).sort_values(ascending=False).pipe(lambda df: pd.concat([df, (df.cumsum() / df.sum() * 100).rename('cumulative_percentage')], axis=1)).astype(int)


,cuit,cumulative_percentage
tipo_societario,,
SOCIEDAD DE RESPONSABILIDAD LIMITADA,420259,38
SOCIEDAD ANONIMA,363875,72
ASOCIACION CIVIL,114779,82
SOCIEDAD POR ACCION SIMPLIFICADA,49655,87
COOPERATIVA,37474,90
SOCIEDAD LEY 19550 CAPITULO I SECCION IV,25362,93
FUNDACION,19263,94
ASOCIACION COOPERADORA,9744,95
COOPERATIVA EFECTORA,8520,96


In [37]:
# soc[['cuit','dom_legal_provincia']].groupby('dom_legal_provincia').count().sort_values(by='cuit',ascending=False)

In [42]:
all_provinces = soc.dom_legal_provincia.unique().tolist()

soc.groupby('dom_legal_provincia')['cuit'].count().reindex(all_provinces, fill_value=0).sort_values(ascending=False).pipe(lambda df: pd.concat([df, (df.cumsum() / df.sum() * 100).rename('cumulative_percentage')], axis=1)).astype(int)


,cuit,cumulative_percentage
dom_legal_provincia,,
CIUDAD AUTONOMA BUENOS AIRES,469473,42
BUENOS AIRES,228264,62
CORDOBA,81339,70
SANTA FE,79333,77
MENDOZA,45863,81
TUCUMAN,20601,83
ENTRE RIOS,16835,84
SALTA,16635,86
CHACO,15827,87


In [36]:
# Assuming 'soc' is your DataFrame
result_df = soc[['cuit', 'dom_legal_provincia']].groupby('dom_legal_provincia').count().sort_values(by='cuit', ascending=False)

# List of all possible provinces
all_provinces = soc.dom_legal_provincia.unique().tolist()

# Reindex to include all provinces with counts, filling missing values with 0
result_df = result_df.reindex(all_provinces, fill_value=0).sort_values(by='cuit', ascending=False)

# Calculate cumulative percentage
result_df['cumulative_percentage'] = np.round(result_df['cuit'].cumsum() / result_df['cuit'].sum() * 100,1)

# Print the resulting DataFrame
result_df

,cuit,cumulative_percentage
dom_legal_provincia,,
CIUDAD AUTONOMA BUENOS AIRES,469473,42.2
BUENOS AIRES,228264,62.7
CORDOBA,81339,70.1
SANTA FE,79333,77.2
MENDOZA,45863,81.3
TUCUMAN,20601,83.2
ENTRE RIOS,16835,84.7
SALTA,16635,86.2
CHACO,15827,87.6


In [28]:
soc[['cuit','tipo_societario']].groupby('tipo_societario').count().sort_values(by='cuit',ascending=False)

,cuit
tipo_societario,
SOCIEDAD DE RESPONSABILIDAD LIMITADA,420259
SOCIEDAD ANONIMA,363875
ASOCIACION CIVIL,114779
SOCIEDAD POR ACCION SIMPLIFICADA,49655
COOPERATIVA,37474
SOCIEDAD LEY 19550 CAPITULO I SECCION IV,25362
FUNDACION,19263
ASOCIACION COOPERADORA,9744
COOPERATIVA EFECTORA,8520


In [19]:
soc.columns

Index(['cuit', 'razon_social', 'fecha_hora_contrato_social', 'tipo_societario',
       'fecha_hora_actualizacion', 'numero_inscripcion',
       'dom_fiscal_provincia', 'dom_fiscal_localidad', 'dom_fiscal_calle',
       'dom_fiscal_numero', 'dom_fiscal_piso', 'dom_fiscal_departamento',
       'dom_fiscal_cp', 'dom_fiscal_estado_domicilio', 'dom_legal_provincia',
       'dom_legal_localidad', 'dom_legal_calle', 'dom_legal_numero',
       'dom_legal_piso', 'dom_legal_departamento', 'dom_legal_cp',
       'dom_legal_estado_domicilio'],
      dtype='object')

In [18]:
soc.head()

,cuit,razon_social,fecha_hora_contrato_social,tipo_societario,fecha_hora_actualizacion,numero_inscripcion,dom_fiscal_provincia,dom_fiscal_localidad,dom_fiscal_calle,dom_fiscal_numero,...,dom_fiscal_cp,dom_fiscal_estado_domicilio,dom_legal_provincia,dom_legal_localidad,dom_legal_calle,dom_legal_numero,dom_legal_piso,dom_legal_departamento,dom_legal_cp,dom_legal_estado_domicilio
0,3.050000e+10,SEGUROS SURA S.A,1912-04-30-10:43,SOCIEDAD ANONIMA,2022-10-24-10:37,NaN,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,"GRIERSON,CECILIA BOULEVARD",255.0,...,1107.0,CONFIRMADO,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,"GRIERSON,CECILIA BOULEVARD",255.0,1,NaN,1107.0,DECLARADO
1,3.050000e+10,BANCO COMERCIAL DE TRES ARROYOS S A,1906-09-01-00:00,SOCIEDAD ANONIMA,2022-06-14-18:59,NaN,BUENOS AIRES,TRES ARROYOS,BETOLAZA,189.0,...,7500.0,INEXISTENTE - DESCONOCIDO,BUENOS AIRES,TRES ARROYOS,AV MORENO,300.0,NaN,NaN,7500.0,NO CONFIRMADO
2,3.050000e+10,BANCO COMERCIAL DEL TANDIL S A,1964-03-21-00:00,SOCIEDAD ANONIMA,2022-09-24-13:02,NaN,BUENOS AIRES,TANDIL,GRAL PINTO,602.0,...,7000.0,DECLARADO,BUENOS AIRES,TANDIL,GRAL PINTO,602.0,NaN,NaN,7000.0,NO CONFIRMADO
3,3.050000e+10,BOSTON COMPAÑIA ARGENTINA DE SEGUROS S A,1925-02-19-11:00,SOCIEDAD ANONIMA,2022-11-15-13:14,NaN,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,SUIPACHA,268.0,...,1008.0,CONFIRMADO,CIUDAD AUTONOMA BUENOS AIRES,CAPITAL FEDERAL,SUIPACHA,268.0,03,NaN,1008.0,DECLARADO
4,3.050000e+10,BANCO DE CREDITO COMERCIAL,1913-03-10-00:00,SOCIEDAD ANONIMA,2022-06-14-18:59,NaN,SANTA FE,SANTA FE,SAN MARTIN,2498.0,...,3000.0,INEXISTENTE - DESCONOCIDO,SANTA FE,SANTA FE,SAN MARTIN,2498.0,NaN,NaN,3000.0,NO CONFIRMADO
